In [204]:
import pyterrier as pt
import os
import pandas as pd

In [205]:
os.environ["JAVA_HOME"] = "C:\\Program Files\\Microsoft\\jdk-11.0.12.7-hotspot\\bin"

In [206]:
if not pt.started():
    pt.init()

PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7



In [207]:
files = pt.io.find_files("./Mails")

indexer = pt.FilesIndexer("E:\Pycharm\IR\Experiment1\mail_index", overwrite=True, verbose=True, blocks=False)
indexref = indexer.index(files)

index = pt.IndexFactory.of(indexref)
print(index.getCollectionStatistics().toString())

21:29:45.441 [main] ERROR org.terrier.structures.indexing.Indexer - Could not rename index
java.io.IOException: Rename of index structure file 'E:\Pycharm\IR\Experiment1\mail_index/data_1.direct.bf' (exists) to 'E:\Pycharm\IR\Experiment1\mail_index/data.direct.bf' (exists) failed - likely that source file is still open. Possible indexing bug?
	at org.terrier.structures.IndexUtil.renameIndex(IndexUtil.java:379)
	at org.terrier.structures.indexing.Indexer.index(Indexer.java:388)
Number of documents: 21
Number of terms: 107
Number of postings: 231
Number of fields: 0
Number of tokens: 345
Field names: []
Positions:   false


In [208]:
queries_dict = {
    1: 'Welcome, Nilesh,Thanks for joining Indian Institute of Information Technology'
}
queries = pd.DataFrame(list(queries_dict.items()), columns=['qid', 'query'])

In [213]:
retr = pt.BatchRetrieve(index, controls = {"wmodel": "TF_IDF"})

retr.setControl("wmodel", "TF_IDF")
retr.setControls({"wmodel": "TF_IDF"})

In [214]:
res = retr.transform(queries)
res

,qid,docid,docno,rank,score,query
0,1,15,d16,0,4.613670,"Welcome, Nilesh,Thanks for joining Indian Inst..."
1,1,14,d15,1,4.218750,"Welcome, Nilesh,Thanks for joining Indian Inst..."
2,1,0,d1,2,3.810985,"Welcome, Nilesh,Thanks for joining Indian Inst..."
3,1,6,d7,3,2.588984,"Welcome, Nilesh,Thanks for joining Indian Inst..."


Score calculation for an query:

In [142]:
import string
import re
import numpy as np
from nltk import word_tokenize
import math

In [177]:
k_1 = 1.2
k_3 = 8
b = 0.75
avgDocLength = 0
no_documents = 21

In [178]:
icon_pattern = r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U0001FB00-\U0001FBFF\U0001FC00-\U0001FCFF\U0001FD00-\U0001FDFF\U0001FE00-\U0001FEFF\U0001FF00-\U0001FFFF]+'
pattern = r'[&#\n*/.,!?(){}<>\[\]"\'=;,@\d]'
symbols_to_remove = r'[!\"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]'
translation_table = str.maketrans(string.punctuation, ' ' * len(string.punctuation))

In [190]:
corpus_path = 'E:/Pycharm/IR/Experiment1/Mails'
ordered_words = [[] for i in range(no_documents)]

In [191]:
for i in range(no_documents):
    with open(f'{corpus_path}/email_{i}.txt', 'r' ,encoding='utf-8') as file:
        text = file.read()
        text = text.split()
        # text = text.translate(translation_table)
        # text = re.sub(icon_pattern, '', text).split(' ')
        for word in text:
            word = str(word).lower()
            word = re.sub(r'[\n]', '', word)
            if word:
                ordered_words[i].append(word)   

In [192]:
for doc in ordered_words:
    avgDocLength += len(doc)
avgDocLength /= no_documents

In [193]:
query = 'Welcome, Nilesh,Thanks for joining Indian Institute of Information Technology'
query2 = query.translate(translation_table).split(' ')
terms = [q.lower() for q in query2 if q]
query = word_tokenize(query.translate(translation_table))
query = [q.lower() for q in query]

In [194]:
tf = [[0 for i in range(len(terms))] for j in range(no_documents)]

for i in range(no_documents):
    for j in range(len(terms)):
        tf[i][j] = ordered_words[i].count(terms[j])

In [195]:
df = [0 for i in range(len(terms))]
i = 0
for term in terms:
    for doc in ordered_words:
        if term in doc: df[i] += 1
    # df[i] /= no_documents
    i += 1

In [196]:
docLengths = [len(doc) for doc in ordered_words]

In [197]:
termFrequencies = [0 for i in range(len(terms))]
for i in range(len(terms)):
    for j in range(no_documents):
        termFrequencies[i] += tf[j][i]

In [198]:
keyFrequencies = [0 for  i in range(len(terms))]
for i in range(len(terms)):
    keyFrequencies[i] = query.count(terms[i]) / len(query)

In [200]:
def score(tf: float, docLength: int, documentFrequency: float, termFrequency:float, keyFrequency: float) -> float:
    robertson_tf = k_1 * tf / (tf + k_1 * (1 - b + b*docLength / avgDocLength ))
    idf = math.log(no_documents/(documentFrequency+ 1))
    return keyFrequency * robertson_tf * idf
    # K = k_1 * ((1 - b) + b * docLength / avgDocLength) + tf
    # return (tf * (k_3 + 1) * keyFrequency / ((k_3 + keyFrequency) * K)) * math.log((no_documents - documentFrequency + 0.5) / (documentFrequency + 0.5))

In [201]:
scores = [0 for i in range(no_documents)]
for i in range(no_documents):
    for j in range(len(terms)):
        scores[i] += score(tf[i][j], docLengths[i], df[j], termFrequencies[j], keyFrequencies[j])
scores

[0.6032201180355741,
 0.2640119618228913,
 0.1118486843749609,
 0.620949371985996,
 0.8131149826492106,
 0.0,
 0.07341425282372888,
 0.0,
 0.06551622380242794,
 0.0,
 0.05726993579515466,
 0.09070607963759321,
 0.09115872567241654,
 0.06425884726926157,
 0.13914621130359753,
 0.06553665626173356,
 0.319449174646459,
 0.030646199967794215,
 0.0,
 0.0,
 0.0]